## Iteration 6

In the sixth iteration, two three improvements were made:

1. The solution field now includes a theoretical explanation (formulas, rules, definitions) before the numbered steps of the solution.

2. Added RAG using math textbooks (grades 5-11) to extract relevant terminology and theory for better context.

3. Model was changed from 4o to more powerful 5.2 (#1 on LMArena)

In [44]:
import json
import os
from glob import glob
from typing import Literal

import jsonlines
import pandas as pd
from dotenv import load_dotenv
from pydantic import BaseModel
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [45]:
pd.set_option('display.max_colwidth', None)

In [46]:
load_dotenv()

True

In [47]:
class MathContent(BaseModel):
    type: Literal["Задача", "Тест"]
    description: str
    solution: str
    answer: str

class JudgmentResult(BaseModel):
    has_theory_section: bool
    has_numbered_solution: bool
    has_valid_answer_format: bool
    has_valid_description_format: bool

In [48]:
with jsonlines.open('dataset.jsonl') as reader:
    dataset = list(reader)

dataset[0]

{'topic': 'Арифметичні дії',
 'type': 'task',
 'description': 'Обчисліть значення виразу: $100 - (15 + 5) \\times 4$.',
 'solution': '1. Виконайте дію в дужках: $15 + 5 = 20$. 2. Виконайте множення отриманого результату на 4: $20 \\times 4 = 80$. 3. Виконайте віднімання від 100: $100 - 80 = 20$.',
 'answer': '20'}

In [49]:
generator_model = ChatOpenAI(model="gpt-5.2", temperature=0.7)
judge_model = ChatOpenAI(model="gpt-5.2", temperature=0.1)

In [50]:
pdf_files = glob('../../books/*.pdf')
persist_path = "../../faiss_index"
embeddings = OpenAIEmbeddings()

if os.path.exists(persist_path):
    vectorstore = FAISS.load_local(persist_path, embeddings, allow_dangerous_deserialization=True)
else:
    documents = []
    for pdf_path in pdf_files:
        loader = PyMuPDFLoader(pdf_path)
        documents.extend(loader.load())

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_documents(documents)
    vectorstore = FAISS.from_documents(chunks, embeddings)
    vectorstore.save_local(persist_path)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [51]:
class MathTutor:
    def __init__(self, model, retriever=None):
        self.model = model
        self.retriever = retriever
        self.parser = PydanticOutputParser(pydantic_object=MathContent)
        
        self.system_prompt = """Ти - вчитель математики. Проаналізуй запит користувача та надай відповідь.

Якщо користувач вказав тип контенту (Задача/Тест) - використай його. Якщо ні - обери відповідний тип самостійно.

Типи контенту:
1. Задача: 
   - description: умова задачі з одним питанням
   - solution: спочатку теоретичне пояснення (формули, правила, означення), потім нумерований список кроків розв'язання
   - answer: число або вираз (без додатковий позначень)

2. Тест:
   - description: умова задачі з чотирма варіантами відповіді (A, B, C, D)
   - solution: спочатку теоретичне пояснення (формули, правила, означення), потім нумерований список кроків для знаходження відповіді
   - answer: одна латинська літера (A, B, C або D)

Відповідь надай у JSON форматі з чотирма полями: type (Задача або Тест), description, solution, answer.
Не обгортай JSON у додаткові спецсимволи."""

        self.examples = [
            {"input": "Тема: Площа круга, Тип: Задача", "output": '{"type": "Задача", "description": "Обчисліть площу круга з радіусом $r = 4$. Прийміть $\\\\pi \\\\approx 3.14$.", "solution": "Площа круга обчислюється за формулою $S = \\\\pi r^2$, де $r$ — радіус круга, а $\\\\pi$ — математична константа, що приблизно дорівнює 3.14. 1. Підставте значення радіуса у формулу: $S = 3.14 \\\\cdot 4^2$. 2. Піднесіть радіус до квадрата: $4^2 = 16$. 3. Виконайте множення: $S = 3.14 \\\\cdot 16 = 50.24$.", "answer": "50.24"}'},
            {"input": "Знайдіть корінь рівняння $2x - 5 = 7$.", "output": '{"type": "Задача", "description": "Знайдіть корінь рівняння $2x - 5 = 7$.", "solution": "Лінійне рівняння з однією змінною розв\\\'язується шляхом виділення невідомої. Для цього переносимо всі члени з невідомою в одну сторону, а вільні члени — в іншу, змінюючи знак на протилежний. 1. Перенесіть вільний член у праву частину: $2x = 7 + 5$. 2. Обчисліть суму: $2x = 12$. 3. Поділіть обидві частини на коефіцієнт при $x$: $x = 12 / 2 = 6$.", "answer": "6"}'},
            {"input": "Чому дорівнює $\\sqrt{144}$? \\n A) 10 \\n B) 11 \\n C) 12 \\n D) 14", "output": '{"type": "Тест", "description": "Чому дорівнює $\\\\sqrt{144}$? \\\\n A) 10 \\\\n B) 11 \\\\n C) 12 \\\\n D) 14", "solution": "Квадратний корінь числа $a$ — це таке невід\\\'ємне число $b$, що $b^2 = a$. Тобто $\\\\sqrt{a} = b$ означає $b \\\\cdot b = a$. 1. Потрібно знайти число, квадрат якого дорівнює 144. 2. Перевіримо варіанти: $10^2 = 100$, $11^2 = 121$, $12^2 = 144$, $14^2 = 196$. 3. Оскільки $12^2 = 144$, правильна відповідь — C.", "answer": "C"}'},
        ]
        
        example_prompt = ChatPromptTemplate.from_messages([
            ("human", "{input}"),
            ("ai", "{output}")
        ])
        
        few_shot_prompt = FewShotChatMessagePromptTemplate(
            examples=self.examples,
            example_prompt=example_prompt
        )
        
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", self.system_prompt),
            few_shot_prompt,
            ("human", "{user_input}")
        ])
        
        self.prompt_with_context = ChatPromptTemplate.from_messages([
            ("system", self.system_prompt + "\n\nВикористай наступний контекст з підручника для формування теоретичного пояснення:\n{context}"),
            few_shot_prompt,
            ("human", "{user_input}")
        ])
        
        self.chain = self.prompt | self.model | self.parser
        self.chain_with_context = self.prompt_with_context | self.model | self.parser

    def invoke(self, user_input: str) -> MathContent:
        if self.retriever:
            docs = self.retriever.invoke(user_input)
            context = "\n\n".join([doc.page_content for doc in docs])
            return self.chain_with_context.invoke({"user_input": user_input, "context": context})
        return self.chain.invoke({"user_input": user_input})

In [52]:
class AcceptanceJudge:
    def __init__(self, model):
        self.model = model
        self.parser = PydanticOutputParser(pydantic_object=JudgmentResult)
        
        self.system_prompt = """Ти - експерт з оцінювання математичного контенту. Оціни наданий контент за критеріями.

Критерії:
1. has_theory_section: чи містить поле solution теоретичне пояснення (формули, правила, означення) перед нумерованим списком кроків
2. has_numbered_solution: чи містить поле solution нумерований список кроків (1. ... 2. ... тощо)
3. has_valid_answer_format: 
   - для типу "Задача": відповідь має бути числом або виразом, або їх комбінацією
   - для типу "Тест": відповідь має бути однією латинською літерою (A, B, C або D)
4. has_valid_description_format:
   - для типу "Задача": опис має містити умову задачі
   - для типу "Тест": опис має містити чотири варіанти відповіді (A, B, C, D)

Відповідь надай у JSON форматі з чотирма булевими полями: has_theory_section, has_numbered_solution, has_valid_answer_format, has_valid_description_format.
Не обгортай JSON у додаткові спецсимволи."""

        self.examples = [
            {"input": 'Тип контенту: Задача\n\nКонтент:\n{"description": "Обчисліть площу круга з радіусом $r = 4$. Прийміть $\\\\pi \\\\approx 3.14$.", "solution": "Площа круга обчислюється за формулою $S = \\\\pi r^2$, де $r$ — радіус круга. 1. Підставте значення радіуса у формулу: $S = 3.14 \\\\cdot 4^2$. 2. Піднесіть радіус до квадрата: $4^2 = 16$. 3. Виконайте множення: $S = 3.14 \\\\cdot 16 = 50.24$.", "answer": "50.24"}', "output": '{"has_theory_section": true, "has_numbered_solution": true, "has_valid_answer_format": true, "has_valid_description_format": true}'},
            {"input": 'Тип контенту: Тест\n\nКонтент:\n{"description": "Знайдіть корінь рівняння $2x - 5 = 7$. \\n A) 1 \\n B) 6 \\n C) 12 \\n D) 2", "solution": "Лінійне рівняння розв\'язується шляхом виділення невідомої. 1. Перенесіть вільний член у праву частину: $2x = 7 + 5$. 2. Обчисліть суму: $2x = 12$. 3. Поділіть на коефіцієнт: $x = 6$. Відповідь B.", "answer": "B"}', "output": '{"has_theory_section": true, "has_numbered_solution": true, "has_valid_answer_format": true, "has_valid_description_format": true}'},
        ]
        
        example_prompt = ChatPromptTemplate.from_messages([
            ("human", "{input}"),
            ("ai", "{output}")
        ])
        
        few_shot_prompt = FewShotChatMessagePromptTemplate(
            examples=self.examples,
            example_prompt=example_prompt
        )
        
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", self.system_prompt),
            few_shot_prompt,
            ("human", "Тип контенту: {content_type}\n\nКонтент:\n{content}")
        ])
        
        self.chain = self.prompt | self.model | self.parser

    def evaluate(self, content: MathContent) -> JudgmentResult:
        content_json = json.dumps(content.model_dump(), ensure_ascii=False)
        return self.chain.invoke({"content_type": content.type, "content": content_json})

In [53]:
tutor = MathTutor(generator_model, retriever)
judge = AcceptanceJudge(judge_model)

def evaluate_input(user_input: str):
    try:
        content = tutor.invoke(user_input)
        tutor_success = True
    except Exception:
        tutor_success = False
        content = None
    
    if content:
        try:
            judgment = judge.evaluate(content)
            judge_success = True
        except Exception:
            judge_success = False
            judgment = None
    else:
        judge_success = None
        judgment = None
    
    display(pd.DataFrame([
        {'field': 'user_input', 'value': user_input},
        {'field': 'tutor_success', 'value': tutor_success},
        {'field': 'tutor_type', 'value': content.type if content else ''},
        {'field': 'tutor_description', 'value': content.description if content else ''},
        {'field': 'tutor_solution', 'value': content.solution if content else ''},
        {'field': 'tutor_answer', 'value': content.answer if content else ''},
        {'field': 'judge_success', 'value': judge_success},
        {'field': 'judge_has_theory_section', 'value': judgment.has_theory_section if judgment else None},
        {'field': 'judge_has_numbered_solution', 'value': judgment.has_numbered_solution if judgment else None},
        {'field': 'judge_has_valid_answer_format', 'value': judgment.has_valid_answer_format if judgment else None},
        {'field': 'judge_has_valid_description_format', 'value': judgment.has_valid_description_format if judgment else None},
    ]))

    return {
        'user_input': user_input[:50] + '...' if len(user_input) > 50 else user_input,
        'tutor_success': tutor_success,
        'tutor_type': content.type if content else '',
        'judge_success': judge_success,
        'judge_has_theory_section': judgment.has_theory_section if judgment else None,
        'judge_has_numbered_solution': judgment.has_numbered_solution if judgment else None,
        'judge_has_valid_answer_format': judgment.has_valid_answer_format if judgment else None,
        'judge_has_valid_description_format': judgment.has_valid_description_format if judgment else None,
    }

results = []

for item in dataset[:5]:
    results.append(evaluate_input(item['topic']))

for item in dataset[:5]:
    results.append(evaluate_input(item['description']))

results = pd.DataFrame(results)

,field,value
0,user_input,Арифметичні дії
1,tutor_success,True
2,tutor_type,Задача
3,tutor_description,Обчисліть значення виразу: $48\div 6+7\cdot 3-5$.
4,tutor_solution,"В арифметичних діях діє порядок виконання: 1) дужки (якщо є), 2) множення і ділення зліва направо, 3) додавання і віднімання зліва направо.\n1. Виконайте ділення: $48\div 6=8$.\n2. Виконайте множення: $7\cdot 3=21$.\n3. Виконайте додавання: $8+21=29$.\n4. Виконайте віднімання: $29-5=24$."
5,tutor_answer,24
6,judge_success,True
7,judge_has_theory_section,True
8,judge_has_numbered_solution,True
9,judge_has_valid_answer_format,True


,field,value
0,user_input,Геометрія
1,tutor_success,True
2,tutor_type,Тест
3,tutor_description,Яке з тверджень є правильним про пряму на площині?\nA) Пряма має два кінці\nB) Пряма є скінченним відрізком\nC) Пряму можна продовжити в обидві сторони необмежено\nD) Пряма складається лише з двох точок
4,tutor_solution,"На площині пряма — це геометрична фігура, яку можна уявно продовжувати в обидві сторони без кінця; отже, вона нескінченна і не має кінців. \n1. Згадаємо означення: пряма не має кінців і може бути продовжена в обидві сторони необмежено.\n2. Порівняємо з варіантами: A і B описують відрізок (скінченну фігуру), D є хибним, бо пряма містить нескінченно багато точок.\n3. Правильне твердження — варіант C."
5,tutor_answer,C
6,judge_success,True
7,judge_has_theory_section,True
8,judge_has_numbered_solution,True
9,judge_has_valid_answer_format,True


,field,value
0,user_input,Спрощення виразів
1,tutor_success,True
2,tutor_type,Задача
3,tutor_description,Спростіть вираз: $3x + 5x - 2(x - 4)$.
4,tutor_solution,Спрощення виразів виконують за допомогою тотожних перетворень: розкриття дужок (використовуємо розподільну властивість множення відносно додавання) та зведення подібних доданків (доданків з однаковою змінною в однаковому степені). 1. Розкриємо дужки: $-2(x-4)=-2x+8$. 2. Запишемо вираз без дужок: $3x+5x-2x+8$. 3. Зведемо подібні доданки: $(3x+5x-2x)=6x$. 4. Отримаємо спрощений вираз: $6x+8$.
5,tutor_answer,6x+8
6,judge_success,True
7,judge_has_theory_section,True
8,judge_has_numbered_solution,True
9,judge_has_valid_answer_format,True


,field,value
0,user_input,Нерівності
1,tutor_success,True
2,tutor_type,Задача
3,tutor_description,"Доведіть нерівність для будь-яких дійсних чисел $a,b,c$:\n$$a^2+b^2+c^2\ge ab+bc+ca.$$"
4,tutor_solution,"Використаємо метод застосування очевидної нерівності: сума квадратів будь-яких дійсних чисел не від’ємна, тобто $x^2\ge 0$.\n\n1. Запишемо очевидні нерівності:\n$$(a-b)^2\ge 0,\quad (b-c)^2\ge 0,\quad (c-a)^2\ge 0.$$\n2. Додамо їх почленно:\n$$(a-b)^2+(b-c)^2+(c-a)^2\ge 0.$$\n3. Розкриємо дужки:\n$$ (a^2-2ab+b^2)+(b^2-2bc+c^2)+(c^2-2ca+a^2)\ge 0.$$\n4. Зведемо подібні доданки:\n$$2(a^2+b^2+c^2)-2(ab+bc+ca)\ge 0.$$\n5. Поділимо обидві частини на 2:\n$$a^2+b^2+c^2-(ab+bc+ca)\ge 0,$$\nтобто\n$$a^2+b^2+c^2\ge ab+bc+ca.$$"
5,tutor_answer,a^2+b^2+c^2\ge ab+bc+ca
6,judge_success,True
7,judge_has_theory_section,True
8,judge_has_numbered_solution,True
9,judge_has_valid_answer_format,True


,field,value
0,user_input,Системи рівнянь
1,tutor_success,True
2,tutor_type,Задача
3,tutor_description,"Розв’яжіть систему рівнянь:\n\[\n\begin{cases}\n2x+y=7,\\\nx-y=1.\n\end{cases}\n\]"
4,tutor_solution,"Система лінійних рівнянь з двома змінними розв’язується способом підстановки або способом додавання. Законність цих способів ґрунтується на рівносильних перетвореннях: якщо замінити рівняння рівносильним або замінити одне з рівнянь сумою (або різницею) двох рівнянь, то отримаємо рівносильну систему (і множина розв’язків не зміниться).\n1. Із другого рівняння виразимо $x$: $x=y+1$.\n2. Підставимо $x=y+1$ у перше рівняння: $2(y+1)+y=7$.\n3. Розкриємо дужки та зведемо подібні: $2y+2+y=7 \Rightarrow 3y=5$.\n4. Знайдемо $y$: $y=\frac{5}{3}$.\n5. Підставимо $y=\frac{5}{3}$ у $x=y+1$: $x=\frac{5}{3}+1=\frac{8}{3}$."
5,tutor_answer,\left(\frac{8}{3};\frac{5}{3}\right)
6,judge_success,True
7,judge_has_theory_section,True
8,judge_has_numbered_solution,True
9,judge_has_valid_answer_format,True


,field,value
0,user_input,Обчисліть значення виразу: $100 - (15 + 5) \times 4$.
1,tutor_success,True
2,tutor_type,Задача
3,tutor_description,Обчисліть значення виразу: $100 - (15 + 5) \times 4$.
4,tutor_solution,"Порядок виконання дій: спочатку обчислюємо дії в дужках, потім множення, потім віднімання. 1. Обчислюємо суму в дужках: $15 + 5 = 20$. 2. Виконуємо множення: $20 \times 4 = 80$. 3. Виконуємо віднімання: $100 - 80 = 20$."
5,tutor_answer,20
6,judge_success,True
7,judge_has_theory_section,True
8,judge_has_numbered_solution,True
9,judge_has_valid_answer_format,True


,field,value
0,user_input,Чому дорівнює сума внутрішніх кутів довільного трикутника? \n A) $90^\circ$ \n B) $180^\circ$ \n C) $270^\circ$ \n D) $360^\circ$
1,tutor_success,True
2,tutor_type,Тест
3,tutor_description,Чому дорівнює сума внутрішніх кутів довільного трикутника?\n A) $90^\circ$\n B) $180^\circ$\n C) $270^\circ$\n D) $360^\circ$
4,tutor_solution,Сума внутрішніх кутів будь-якого трикутника є сталою і дорівнює $180^\circ$ (це властивість трикутника в евклідовій геометрії).\n1. Згадаємо властивість: $\angle A+\angle B+\angle C=180^\circ$ для будь-якого трикутника.\n2. Порівняємо з варіантами відповідей.\n3. Значення $180^\circ$ відповідає варіанту B.
5,tutor_answer,B
6,judge_success,True
7,judge_has_theory_section,True
8,judge_has_numbered_solution,True
9,judge_has_valid_answer_format,True


,field,value
0,user_input,"Спростіть вираз, використовуючи формулу різниці квадратів: $(x - 3)(x + 3)$."
1,tutor_success,True
2,tutor_type,Задача
3,tutor_description,"Спростіть вираз, використовуючи формулу різниці квадратів: $(x - 3)(x + 3)$."
4,tutor_solution,"Використовуємо формулу різниці квадратів: $(a-b)(a+b)=a^2-b^2$. 1. Порівняємо $(x-3)(x+3)$ з $(a-b)(a+b)$: тут $a=x$, $b=3$. 2. Застосуємо формулу: $(x-3)(x+3)=x^2-3^2$. 3. Обчислимо квадрат числа 3: $x^2-9$."
5,tutor_answer,x^2-9
6,judge_success,True
7,judge_has_theory_section,True
8,judge_has_numbered_solution,True
9,judge_has_valid_answer_format,True


,field,value
0,user_input,Який проміжок є розв'язком нерівності $x > 5$? \n A) $(-\infty; 5)$ \n B) $[5; +\infty)$ \n C) $(5; +\infty)$ \n D) $(-\infty; 5]$
1,tutor_success,True
2,tutor_type,Тест
3,tutor_description,Який проміжок є розв'язком нерівності $x > 5$?\n A) $(-\infty; 5)$\n B) $[5; +\infty)$\n C) $(5; +\infty)$\n D) $(-\infty; 5]$
4,tutor_solution,"Нерівність вигляду $x>a$ означає, що розв’язками є всі числа, строго більші за $a$. Тоді точка $a$ не входить у проміжок, і використовується кругла дужка.\n1. Для $x>5$ беремо всі числа, що більші за 5.\n2. Число 5 не включається (бо знак строгий "">"").\n3. Отже, множина розв’язків: $(5; +\infty)$, що відповідає варіанту C."
5,tutor_answer,C
6,judge_success,True
7,judge_has_theory_section,True
8,judge_has_numbered_solution,True
9,judge_has_valid_answer_format,True


,field,value
0,user_input,Розв'яжіть систему рівнянь: $\begin{cases} x + y = 10 \\ x - y = 4 \end{cases}$
1,tutor_success,True
2,tutor_type,Задача
3,tutor_description,Розв'яжіть систему рівнянь: $\begin{cases} x + y = 10 \\ x - y = 4 \end{cases}$
4,tutor_solution,"Систему лінійних рівнянь із двома змінними можна розв’язувати методом підстановки: з одного рівняння виражають одну змінну через іншу і підставляють у друге рівняння, отримуючи лінійне рівняння з однією змінною. \n1. З першого рівняння виразимо $y$ через $x$: $y = 10 - x$.\n2. Підставимо цей вираз у друге рівняння: $x - (10 - x) = 4$.\n3. Розкриємо дужки: $x - 10 + x = 4$.\n4. Зведемо подібні доданки: $2x - 10 = 4$.\n5. Перенесемо $-10$ вправо: $2x = 14$.\n6. Поділимо на 2: $x = 7$.\n7. Знайдемо $y$: $y = 10 - 7 = 3$."
5,tutor_answer,(7;3)
6,judge_success,True
7,judge_has_theory_section,True
8,judge_has_numbered_solution,True
9,judge_has_valid_answer_format,True


In [54]:
display(results)

bool_columns = ['tutor_success', 'judge_success', 'judge_has_theory_section', 'judge_has_numbered_solution', 'judge_has_valid_answer_format', 'judge_has_valid_description_format']
failures = results[~results[bool_columns].all(axis=1)]

print(f"Failures: {len(failures)} / {len(results)}")
if not failures.empty:
    display(failures)

,user_input,tutor_success,tutor_type,judge_success,judge_has_theory_section,judge_has_numbered_solution,judge_has_valid_answer_format,judge_has_valid_description_format
0,Арифметичні дії,True,Задача,True,True,True,True,True
1,Геометрія,True,Тест,True,True,True,True,True
2,Спрощення виразів,True,Задача,True,True,True,True,True
3,Нерівності,True,Задача,True,True,True,True,True
4,Системи рівнянь,True,Задача,True,True,True,True,True
5,Обчисліть значення виразу: $100 - (15 + 5) \times ...,True,Задача,True,True,True,True,True
6,Чому дорівнює сума внутрішніх кутів довільного три...,True,Тест,True,True,True,True,True
7,"Спростіть вираз, використовуючи формулу різниці кв...",True,Задача,True,True,True,True,True
8,Який проміжок є розв'язком нерівності $x > 5$? \n A...,True,Тест,True,True,True,True,True
9,Розв'яжіть систему рівнянь: $\begin{cases} x + y =...,True,Задача,True,True,True,True,True


Failures: 0 / 10
